In [ ]:
# Basic imports
import numpy as np
import geopandas as gpd
from pathlib import Path

# Library imports
from geograypher.cameras.derived_cameras import MetashapeCameraSet
from geograypher.meshes import TexturedPhotogrammetryMesh
from geograypher.utils.visualization import show_segmentation_labels
from geograypher.constants import (
    EXAMPLE_LABELS_FILENAME,
    EXAMPLE_MESH_FILENAME,
    EXAMPLE_CAMERAS_FILENAME,
    EXAMPLE_DTM_FILE,
    EXAMPLE_IMAGE_FOLDER,
    EXAMPLE_LABELED_MESH_FILENAME,
    EXAMPLE_RENDERED_LABELS_FOLDER,
    EXAMPLE_IDS_TO_LABELS,
    EXAMPLE_LABEL_COLUMN_NAME,
)

# Set constants
You should be able to define most of the behavior from these constants

In [ ]:
## Parameters to control the outputs
# Repeat the labeling process
RETEXTURE = True
# Points less than this height above the DTM are considered ground
# Something is off about the elevation between the mesh and the DTM, this should be a threshold in meters above ground
HEIGHT_ABOVE_GROUND_THRESH = 2
# The image is downsampled to this fraction for accelerated rendering
RENDER_IMAGE_SCALE = 1
# Portions of the mesh within this distance of the labels are used for rendering
MESH_BUFFER_RADIUS_METER = 20
# Cameras within this radius of the annotations are used for training
CAMERAS_BUFFER_RADIUS_METERS = 50
# Downsample target
DOWNSAMPLE_TARGET = 1

## Define the inputs
# The mapping between integer class IDs and string class names
IDS_TO_LABELS = EXAMPLE_IDS_TO_LABELS
# The input labels
LABELS_FILENAME = EXAMPLE_LABELS_FILENAME
# Render data from this column in the geofile to each image
LABEL_COLUMN_NAME = EXAMPLE_LABEL_COLUMN_NAME
# The mesh exported from Metashape
MESH_FILENAME = EXAMPLE_MESH_FILENAME
# The camera file exported from Metashape
CAMERAS_FILENAME = EXAMPLE_CAMERAS_FILENAME
# The digital elevation map exported by Metashape
DTM_FILE = EXAMPLE_DTM_FILE
# The image folder used to create the Metashape project
IMAGE_FOLDER = EXAMPLE_IMAGE_FOLDER

## Define the intermediate results
# Processed geo file
# Where to save the mesh after labeling
LABELED_MESH_FILENAME = EXAMPLE_LABELED_MESH_FILENAME
# Where to save the rendering label images
RENDER_FOLDER = EXAMPLE_RENDERED_LABELS_FOLDER

# Geospatial processing

Preprocess the geospatial data to be as expected.

In [ ]:
# Load the data
gdf = gpd.read_file(LABELS_FILENAME)
# Show
gdf.plot(LABEL_COLUMN_NAME, legend=True, vmin=-0.5, vmax=9.5, cmap="tab10")

# Load the mesh and read texture from geopolygon

In [ ]:
# Create a labeled version of the mesh from the field data
# if not present or requested
if not Path(LABELED_MESH_FILENAME).is_file() or RETEXTURE:
    # Load the downsampled mesh and apply the texture from the vector file
    mesh = TexturedPhotogrammetryMesh(
        MESH_FILENAME,
        downsample_target=DOWNSAMPLE_TARGET,
        texture=LABELS_FILENAME,
        ROI=LABELS_FILENAME,
        ROI_buffer_meters=MESH_BUFFER_RADIUS_METER,
        IDs_to_labels=IDS_TO_LABELS,
        texture_column_name=LABEL_COLUMN_NAME,
        transform_filename=CAMERAS_FILENAME,
    )
    # Get the vertex textures from the mesh
    texture_verts = mesh.get_texture(
        request_vertex_texture=True, try_verts_faces_conversion=False
    )
    mesh.label_ground_class(
        DTM_file=DTM_FILE,
        height_above_ground_threshold=HEIGHT_ABOVE_GROUND_THRESH,
        only_label_existing_labels=True,
        ground_class_name="GROUND",
        ground_ID=np.nan, # This means that no ground label will be included
        set_mesh_texture=True,
    )

    mesh.save_mesh(LABELED_MESH_FILENAME, save_vert_texture=True)
else:
    mesh = TexturedPhotogrammetryMesh(
        LABELED_MESH_FILENAME, transform_filename=CAMERAS_FILENAME
    )

# Load a set of cameras and subset them to the region around annotations

In [ ]:
# Create camera set
camera_set = MetashapeCameraSet(CAMERAS_FILENAME, IMAGE_FOLDER)
# Extract cameras near the training data
training_camera_set = camera_set.get_subset_ROI(
    ROI=LABELS_FILENAME, buffer_radius=CAMERAS_BUFFER_RADIUS_METERS
)
# Show the camera set
training_camera_set.vis(force_xvfb=True, frustum_scale=0.5)

# Show the mesh

In [ ]:
# You can include the camera set, but it's cleaner without it
mesh.vis(camera_set=None, force_xvfb=True)

# Render the labels onto the images

In [ ]:
mesh.save_renders(
    camera_set=training_camera_set,
    render_image_scale=RENDER_IMAGE_SCALE,
    save_native_resolution=True,
    output_folder=RENDER_FOLDER,
)

# Show some of the rendered labels

In [ ]:
show_segmentation_labels(
    label_folder=RENDER_FOLDER,
    image_folder=IMAGE_FOLDER,
    num_show=10,
    label_suffix=".png",
)